In [1]:
use Modern::Perl;
use Math::Vector::Real;

## Ability to work with vectors is prerequisite to carrying out structural analysis of biological molecules.
At a given temperature, pressure, and molar constitution $G(T,P,n_1,n_2 ... )$, a biological molecule (or collection of molecules) has a space of available conformations or configurations that may be large ($S=k\ln W$).  If you were to carry out a molecular dynamics simulation (which we will come back to later), you would most likely prepare your system using a single structure provided by X-ray crystallography or NMR spectroscopy, solvate your system in water (maybe with salt ions), and then add some heat so that it wiggles about.  

#### Mental exercise: 
Before going an further.  Take look at the PDB file for a mutant of T4 Lysozyme [PDBID: 1L54](http://www.rcsb.org/pdb/files/1L54.pdb). Find the lines of coordinates that also contain additional meta data (what is this "metadata").  Load PDB 1l54 into VMD and imagine it wiggling about.  Imagine it unfolding.

### Lab exercise: Fun with coordinates and vectors
You can represent the structure of a biomolecule as a collection of atoms, each with it set of cartesian coordinates (x, y, and z).  In a PDB file, each atom has some extra information associated with it (e.g. the atom type, the residue to which it belongs, etc).  All this extra information is where the story telling comes in, all the rigorous analysis is in the handling of the coordinates, which can be treated as vectors.

In this notebook, you will work with the coordinates of benzene (C6H6) to learn how to carry out analysis with vectors. First, let's strip down all the information to represent benzene as a set of vectors (all in Angstroms):
         
     V(x,                y,               z)
 0.    V( 0.00000,        1.40272,        0.00000) 
 1.    V(-1.21479,        0.70136,        0.00000)
 2.    V(-1.21479,       -0.70136,        0.00000)
 3.    V( 0.00000,       -1.40272,        0.00000)
 4.    V( 1.21479,       -0.70136,        0.00000)
 5.    V( 1.21479,        0.70136,        0.00000)
 6.    V( 0.00000,        2.49029,        0.00000)
 7.    V(-2.15666,        1.24515,        0.00000)
 8.    V(-2.15666,       -1.24515,        0.00000)
 9.    V( 0.00000,       -2.49029,        0.00000)
 10.   V( 2.15666,       -1.24515,        0.00000)
 11.   V( 2.15666,        1.24515,        0.00000)

That is, the coordinates of each atom are the x,y and z components of a vector extending from the origin (0,0,0) to the center of the atom.  Use the information above to answer these questions:

    1. How is the ring oriented?
    2. Where is the center of the ring?
    3. Which vectors correspond to H atoms and which to C atoms?
    4. What is the origin for the coordinates?
    5. What would happen to the coordinates if you shifted the molecule from the origin in the Z direction by 10 angstroms?
    
Vectors are fun to work with. A vector has magnitude and direction. You should be able to answer the above questions using vector subtraction and addition, which we will review here. All of the above vectors can be written in a general form:

$\vec{R} =  R_x \hat{x} + R_y \hat{y} + R_z \hat{z} $

where the vector ($\vec{R}$) is written using scaled unit vectors pointing along the axes of the coordinate system ($\hat{x},\hat{y}, \hat{z}$). A unit vector has a magnitude of 1.0, and it labeled here with the ^ symbol.  For example, $\hat{R}$ would be a unit vector pointing the same direction as $\vec{R}$.  The three unit vectors ($\hat{x},\hat{y}, \hat{z}$) provide the basis vectors of the system: any vector in this space can be represented by the addition of the scaled unit vectors! The scaling of the unit vectors is accomplished from the coefficients ($R_x, R_y, R_z$). Does this make sense?  Draw some two-dimensional vectors to convince yourself.

Vector subtraction,

$\vec{R_1} - \vec{R_2} =  (R_{x1} - R_{x2})\hat{x} + (R_{y1} - R_{y2})\hat{y} + (R_{z1}- R_{z2})\hat{z} $

and addition, 

$\vec{R_1} + \vec{R_2} =  (R_{x1} + R_{x2})\hat{x} + (R_{y1} + R_{y2})\hat{y} + (R_{z1} + R_{z2})\hat{z} $

is accomplished using the coefficients of the unit vectors.   Thus, the coordinates of benzene, repr

o let's start with those using the vectors 0. and 1. from above,

$\vec{V}_0 =  0.00000 \hat{x} + 1.40272\hat{y} + 0.00000 \hat{z} $

$\vec{V}_1 = -1.21479 \hat{x} + 0.70136\hat{y} + 0.00000 \hat{z} $

Subracting $V_1$ from $V_0$, we find:

$\vec{V}_{0,1} = 1.21479 \hat{x} + 0.70136\hat{y} + 0.00000 \hat{z}$

The vector $\vec{V}_{0,1}$ is that which connects the two points.  This would correspond to the bond vector between the two atoms if they were bonded!  Can we determine if the two vectors are bonded?  Not really because we no longer have any atomic information.  However, we do know this corresponds to benzene so we can hypothesize what atoms these correspond to! In Benzene, carbons are around 1.4 angstroms apart, due to conjugation.  Single bonded carbons around around 1.5 angstroms apart.  A carbon-hydrogen bond is closer to 1 angstrom.  So the question becomes, how to determine the magnitude of $\vec{V}_{0,1}$? The magnitude will give us the distance between the points.  The magnitude of a vector can be calculated from the square root of the dot product (also called scalar product) of the vector with itself:

$\vec{V}_{0,1}\cdot \vec{V}_{0,1} =  1.21479^2 + 0.70136^2 + 0^2$

Verify that this is correct by expanding out the multiplication and using the fact that the basis vectors are othonormal:

$\hat{x}\cdot \hat{x} = 1 = \hat{y}\cdot \hat{y} = \hat{z}\cdot \hat{z}$

$\hat{x}\cdot \hat{y} = 0 = \hat{x}\cdot \hat{z} = \hat{y}\cdot \hat{z}$

$\sqrt{\vec{V}_{0,1}\cdot \vec{V}_{0,1}} = 1.402$ angstroms.

I think now you have all you need to determine all your H atoms and C atom vectors.  Let's do it with Perl using the [Math::Vector::Real] (https://metacpan.org/pod/Math::Vector::Real) library that is loaded above.  You can load the documentation by opening the Math::Vector::Real clicking the link!

### Some example code corresponding to the example above, which you can copy and paste into new cells.

In [2]:
# copy and paste two vectors V_0 and V_1 from above
my $v0 = V( 0.00000, 1.40272, 0.00000);
my $v1 = V(-1.21479, 0.70136, 0.00000);

#find $v01 as above
my $v01 = $v0 - $v1;
say 'displacement vector: ', $v01;

#find the dot product of $v01 with itself
say 'dot product: ', $v01*$v01;

#take the square root of the dot product to find the magnitude

say 'distance: ', sqrt($v01*$v01);

# or we can do it all in one move! 
say 'distance:', abs($v0-$v1);


displacement vector: {1.21479, 0.70136, 0}
dot product: 1.9676205937
distance: 1.40271900026342
distance:1.40271900026342


1


#### Exercise. attempt calculate the distance of displacement vectors to determine which atom is which.
The first step is to collect all the information into a set of variables that we can use over and over:

In [3]:
my $v0  = V( 0.00000,  1.40272, 0.00000);
my $v1  = V(-1.21479,  0.70136, 0.00000);
my $v2  = V(-1.21479, -0.70136, 0.00000);
my $v3  = V( 0.00000, -1.40272, 0.00000);
my $v4  = V( 1.21479, -0.70136, 0.00000);
my $v5  = V( 1.21479,  0.70136, 0.00000);
my $v6  = V( 0.00000,  2.49029, 0.00000);
my $v7  = V(-2.15666,  1.24515, 0.00000);
my $v8  = V(-2.15666, -1.24515, 0.00000);
my $v9  = V( 0.00000, -2.49029, 0.00000);
my $v10 = V( 2.15666, -1.24515, 0.00000);
my $v11 = V( 2.15666,  1.24515, 0.00000);

{2.15666, 1.24515, 0}


Next, since we are going to want to loop over all the vectors in our distance measurement, we assign them all to an array @vecs :

In [4]:
my @vecs = ($v0, $v1, $v2, $v3, $v4, $v5, $v6, $v7, $v8, $v9, $v10, $v11);

{0, 1.40272, 0}{-1.21479, 0.70136, 0}{-1.21479, -0.70136, 0}{0, -1.40272, 0}{1.21479, -0.70136, 0}{1.21479, 0.70136, 0}{0, 2.49029, 0}{-2.15666, 1.24515, 0}{-2.15666, -1.24515, 0}{0, -2.49029, 0}{2.15666, -1.24515, 0}{2.15666, 1.24515, 0}


We are now ready to go.  Let's measure the distance of all atoms from the origin:

In [5]:
say abs($_) foreach @vecs;

1.40272
1.40271900026342
1.40271900026342
1.40272
1.40271900026342
1.40271900026342
2.49029
2.49029734732622
2.49029734732622
2.49029
2.49029734732622
2.49029734732622


At this point we have a pretty good idea that the origin is near the middle of the molecule, the carbons are the first 6 atoms and the hydrogens are the next 6.  Let's not stop here.  Let's rewrite that loop above in two different ways, which will be useful for different situations:
    1.

In [11]:
say "first one, again:";
say abs($_) foreach @vecs;

say "\n more explicit loop over elements of the array";
foreach my $v (@vecs){
    say abs($v);
}

say "\n loop over the indices 0 .. 11 of the array, access the elements of the array by index";

foreach my $iv (0 .. $#vecs){
    say abs($vecs[$iv]);
}

first one, again:
1.40272
1.40271900026342
1.40271900026342
1.40272
1.40271900026342
1.40271900026342
2.49029
2.49029734732622
2.49029734732622
2.49029
2.49029734732622
2.49029734732622

 more explicit loop over elements of the array
1.40272
1.40271900026342
1.40271900026342
1.40272
1.40271900026342
1.40271900026342
2.49029
2.49029734732622
2.49029734732622
2.49029
2.49029734732622
2.49029734732622

 loop over the indices 0 .. 11 of the array, access the elements of the array by index
1.40272
1.40271900026342
1.40271900026342
1.40272
1.40271900026342
1.40271900026342
2.49029
2.49029734732622
2.49029734732622
2.49029
2.49029734732622
2.49029734732622


From the above you can see "that there is more than one way to do it" (TIMTOWDI).  The first is nice and compact but difficult to do more complex work.  The second one is nice unless you need the indices, which we will down below.  For each loop you can see that the foreach function takes a list of values, which can be passed as an array, and sends them into the loop with a defined variable \$v, \$iv, or an implicit variable $_. Implicit variables take some getting used to, but they are useful when learned.   

Next, let's use a loop to calculate the distance between \$v0 and all those in @vecs:

In [12]:
say abs($_-$v0) foreach @vecs;

0
1.40271900026342
2.42958173159497
2.80544
2.42958173159497
1.40271900026342
1.08757
2.16240853228524
3.41502531359579
3.89301
3.41502531359579
2.16240853228524


\$v0 is carbon that is bound to carbons at \$v1 and \$v5 and a hydrogen at \$v6.

####  Exercise.  Use scripting to calculate the distance between each pair of vectors! 
How many are there? Count them up!  You can make an array of all the vectors, as I did below with repeats of the first two defined above.  You will need to use two loops (one nested within the other) to do this. I started this for you below with one loop).

In [14]:
my $count = 0;
foreach my $iv (0 .. $#vecs){
    foreach my $jv ($iv+1 .. $#vecs){
        $count++;
        printf("%5i%5i%10.3f\n",$iv, $jv, abs($vecs[$iv]-$vecs[$jv]));
    }
}
say $count;

    0    1     1.403
    0    2     2.430
    0    3     2.805
    0    4     2.430
    0    5     1.403
    0    6     1.088
    0    7     2.162
    0    8     3.415
    0    9     3.893
    0   10     3.415
    0   11     2.162
    1    2     1.403
    1    3     2.430
    1    4     2.805
    1    5     2.430
    1    6     2.162
    1    7     1.088
    1    8     2.162
    1    9     3.415
    1   10     3.893
    1   11     3.415
    2    3     1.403
    2    4     2.430
    2    5     2.805
    2    6     3.415
    2    7     2.162
    2    8     1.088
    2    9     2.162
    2   10     3.415
    2   11     3.893
    3    4     1.403
    3    5     2.430
    3    6     3.893
    3    7     3.415
    3    8     2.162
    3    9     1.088
    3   10     2.162
    3   11     3.415
    4    5     1.403
    4    6     3.415
    4    7     3.893
    4    8     3.415
    4    9     2.162
    4   10     1.088
    4   11     2.162
    5    6     2.162
    5    7     3.415
    5    8   

1


#### Exercise. Adjust your loops above to create a new array of distances.  Then use a grep to select for the distances that are less than 1.6 angstroms!  

In [15]:
my @dists;
foreach my $iv (0 .. $#vecs){
    foreach my $jv ($iv+1 .. $#vecs){
        push @dists, abs($vecs[$iv]-$vecs[$jv]);
    }
}

say foreach grep {$_ < 1.6} @dists;

1.40271900026342
1.40271900026342
1.08757
1.40272
1.08757834706287
1.40271900026342
1.08757834706287
1.40271900026342
1.08757
1.40272
1.08757834706287
1.08757834706287


#### Exercise. define a displacement vector that moves the molecule 10 angstroms in the Z direction.  Carry out this translation for each atom and then verify that it did nothing to the distances between the "atoms".

In [18]:
my $zdisp = V(0,0,10);
$vecs[$_] += $zdisp foreach 0 .. $#vecs;

say abs($_) foreach @vecs;

say abs($vecs[$_]-$vecs[0]) foreach 0 .. $#vecs;

30.0327758190681
30.0327757723741
30.0327757723741
30.0327758190681
30.0327757723741
30.0327757723741
30.1031816305868
30.1031822383963
30.1031822383963
30.1031816305868
30.1031822383963
30.1031822383963
0
1.40271900026342
2.42958173159497
2.80544
2.42958173159497
1.40271900026342
1.08757
2.16240853228524
3.41502531359579
3.89301
3.41502531359579
2.16240853228524


Congratulations!  In that last loop, we had to use the zeroth element of the @vecs array rather than $v0 as earlier, because $v0 did not change when we displaced all the vectors in @vecs!  